In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType
from pyspark.sql import functions as F

In [48]:
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

In [49]:
print(spark.version)

3.5.5


In [ ]:
myschema = StructType([StructField("id", IntegerType(), True),
                        StructField("name", StringType(), True),
                        StructField("age", IntegerType(), True),
                        StructField("friends", IntegerType(), True)
])

In [ ]:
FordA= spark.read.csv("FordA.csv", header=True,inferSchema=True)


In [ ]:
FordA.printSchema()

root
 |-- Timestamp: string (nullable = true)
 |-- C1: double (nullable = true)
 |-- C2: double (nullable = true)
 |-- C3: double (nullable = true)
 |-- C4: double (nullable = true)
 |-- C5: double (nullable = true)
 |-- C6: double (nullable = true)
 |-- C7: double (nullable = true)
 |-- C8: double (nullable = true)
 |-- C9: double (nullable = true)
 |-- C10: double (nullable = true)
 |-- C11: double (nullable = true)
 |-- C12: double (nullable = true)
 |-- C13: double (nullable = true)
 |-- C14: double (nullable = true)
 |-- C15: double (nullable = true)
 |-- C16: double (nullable = true)
 |-- C17: double (nullable = true)
 |-- C18: double (nullable = true)
 |-- C19: double (nullable = true)
 |-- C20: double (nullable = true)
 |-- C21: double (nullable = true)
 |-- C22: double (nullable = true)
 |-- C23: double (nullable = true)
 |-- C24: double (nullable = true)
 |-- C25: double (nullable = true)
 |-- C26: double (nullable = true)
 |-- C27: double (nullable = true)
 |-- C28: double (

In [ ]:
FordA.show(5, False)

+----------------+-----------+------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-----------+------------+----------+----------+-----------+-----------+----------+-----------+----------

In [ ]:
average = FordA.groupBy("Target").agg(F.count("C1"))

In [ ]:
average.show()

+------+---------+
|Target|count(C1)|
+------+---------+
|    -1|     2527|
|     1|     2394|
+------+---------+



In [ ]:
df=spark.createDataFrame

In [ ]:
df_working=FordA.filter(FordA["Target"]== 1)

In [ ]:
df_working.show(5, False)

+----------------+-----------+----------+------------+-----------+-----------+-----------+------------+----------+----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+------------+-----------+-----------+-----------+-----------+----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----------+-----------+-----------+-----------+----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+----------+----------+----------+-----------+-----------+-----

In [ ]:
df_notworking=FordA.filter(FordA["Target"]== -1)

In [ ]:
df_notworking.show(5, False)

+----------------+-----------+------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+-----------+-----------+----------+----------+-----------+-----------+-----------+-----------+-----------+----------+----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+------------+-----------+----------+-----------+-----------+----------+-----------+-----------+

# ML

In [61]:
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import MinMaxScaler, Normalizer
from pyspark.ml.feature import ChiSqSelector, ChiSqSelectorModel
from pyspark.ml.feature import PCA, PCAModel

In [63]:
train_df,test_df= FordA.randomSplit([0.8, 0.2], seed=42)
train_df.show(5, False)
test_df.show(5, False)

+----------------+-----------+------------+-----------+-----------+-----------+-----------+------------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+------------+-----------+-----------+-----------+-----------+----------+-----------+-----------+------------+------------+------------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+------------+----------+----------+----------+-----------+----------+-----------+---------

In [69]:
train_df.count(), test_df.count()

(3977, 944)

In [73]:
train_df.columns

['Timestamp',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35',
 'C36',
 'C37',
 'C38',
 'C39',
 'C40',
 'C41',
 'C42',
 'C43',
 'C44',
 'C45',
 'C46',
 'C47',
 'C48',
 'C49',
 'C50',
 'C51',
 'C52',
 'C53',
 'C54',
 'C55',
 'C56',
 'C57',
 'C58',
 'C59',
 'C60',
 'C61',
 'C62',
 'C63',
 'C64',
 'C65',
 'C66',
 'C67',
 'C68',
 'C69',
 'C70',
 'C71',
 'C72',
 'C73',
 'C74',
 'C75',
 'C76',
 'C77',
 'C78',
 'C79',
 'C80',
 'C81',
 'C82',
 'C83',
 'C84',
 'C85',
 'C86',
 'C87',
 'C88',
 'C89',
 'C90',
 'C91',
 'C92',
 'C93',
 'C94',
 'C95',
 'C96',
 'C97',
 'C98',
 'C99',
 'C100',
 'C101',
 'C102',
 'C103',
 'C104',
 'C105',
 'C106',
 'C107',
 'C108',
 'C109',
 'C110',
 'C111',
 'C112',
 'C113',
 'C114',
 'C115',
 'C116',
 'C117',
 'C118',
 'C119',
 'C120',
 'C121',
 'C12

In [81]:
vecAssembler= VectorAssembler(inputCols=train_df.columns[1:-1], outputCol="Features")

In [86]:
vecAssembler

VectorAssembler_f842aab56a53

In [92]:
vecTrainDf=vecAssembler.transform(train_df).select("Features","Target")
vecTrainDf.show(5, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [98]:
kmeans= KMeans(k=2, seed=1, featuresCol=vecAssembler.getOutputCol(), predictionCol="cluster")
model=kmeans.fit(vecTrainDf)
print("Model is trained")

Model is trained


Print Cetroid for each cluster

In [99]:
centers=model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 2.85728547e-02 -1.13438752e-02 -5.28257705e-02 -9.21732005e-02
 -1.25381229e-01 -1.49112704e-01 -1.62345060e-01 -1.64054638e-01
 -1.54067093e-01 -1.34737617e-01 -1.07816381e-01 -7.51528571e-02
 -3.88821478e-02 -3.63273658e-04  3.83187322e-02  7.58544554e-02
  1.10664396e-01  1.41862972e-01  1.67247269e-01  1.85320514e-01
  1.94330182e-01  1.91966576e-01  1.76624285e-01  1.47947772e-01
  1.07119877e-01  5.65709624e-02 -3.01307665e-04 -5.88900036e-02
 -1.14333721e-01 -1.62256617e-01 -1.99507166e-01 -2.24313156e-01
 -2.36172144e-01 -2.34685013e-01 -2.19863401e-01 -1.92457980e-01
 -1.53060652e-01 -1.02556688e-01 -4.28725230e-02  2.29121628e-02
  9.01213222e-02  1.53859629e-01  2.09085380e-01  2.50683207e-01
  2.75172254e-01  2.80048169e-01  2.65607827e-01  2.33348370e-01
  1.87676505e-01  1.32184627e-01  7.18618806e-02  1.06419169e-02
 -4.77306107e-02 -1.01244653e-01 -1.48493404e-01 -1.88795256e-01
 -2.21293685e-01 -2.45297593e-01 -2.58659799e-01 -2.59908871e-01
 -2.472

In [101]:
predictions=model.transform(vecTrainDf)
predictions.groupBy("cluster").count().orderBy("cluster").show()
predictions.select("Features","Target","cluster").show(50, False)

+-------+-----+
|cluster|count|
+-------+-----+
|      0| 2033|
|      1| 1944|
+-------+-----+

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Build KNN

In [102]:
ford_array= FordA.toPandas().to_numpy()

In [103]:
ford_array.shape

(4921, 502)

In [104]:
ford_array

array([['01/01/2025 00:00', -0.79717168, -0.66439208, ..., -1.0737958,
        -1.5643427, -1],
       ['01/01/2025 00:01', 0.80485472, 0.63462859, ..., -0.56044294,
        -0.31908642, 1],
       ['01/01/2025 00:02', 0.7279851, 0.11128392, ..., 0.33940042,
        0.25539062, -1],
       ...,
       ['04/01/2025 09:58', 0.71008362, 0.59397882, ..., -0.11782239,
        -0.18909413, 1],
       ['04/01/2025 09:59', 0.006847062, -0.14062427, ..., -1.0841075,
        -1.109963, 1],
       ['04/01/2025 10:00', -0.54135529, -0.24172258, ..., -0.90080431,
        -1.778341, 1]], dtype=object)

In [ ]:
def kNearestNeighbours(pdataset,point,k):
    results={}
    for point_item in pdataset:
        euclidean_distance=math.sqrt(np.sum(np.subtract(point_item,point)*np.subtract(point_item,point)))
        if len(results)<k:
            results[euclidean_distance]=point_item
        else:
            for max_key in sorted(results.keys(),reverse=True):
                if max_key>euclidean_distance:
                    results[euclidean_distance]=point_item
                    results.pop
                break

    return results

IndentationError: expected an indented block after 'for' statement on line 3 (2062973289.py, line 4)